In [1]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 935.6/935.6 kB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 105.1 MB/s eta 0:00:00


In [2]:
import pennylane as qml
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, BoundaryNorm
import sys

# Setting our constants
sys.path.append('..')

In [3]:
!wget https://raw.githubusercontent.com/aifactory-team/AFCompetition/main/9245/train_X.npy
!wget https://raw.githubusercontent.com/aifactory-team/AFCompetition/main/9245/train_y.npy

--2026-01-15 14:43:56--  https://raw.githubusercontent.com/aifactory-team/AFCompetition/main/9245/train_X.npy
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32896 (32K) [application/octet-stream]
Saving to: ‘train_X.npy’

train_X.npy         100%[===================>]  32.12K  --.-KB/s    in 0.009s  

2026-01-15 14:43:56 (3.34 MB/s) - ‘train_X.npy’ saved [32896/32896]

--2026-01-15 14:43:56--  https://raw.githubusercontent.com/aifactory-team/AFCompetition/main/9245/train_y.npy
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 256 [

In [4]:
train_X = np.load("train_X.npy")
train_y = np.load("train_y.npy")

In [5]:
!pip install --upgrade pennylane pennylane-lightning pennylane-lightning-gpu --quiet




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.3/913.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 581.2/581.2 MB 95.9 MB/s eta 0:00:01
ERROR: Operation cancelled by user


In [6]:
import pennylane as qml
import torch
import torch.nn as nn
import torch.optim as optim
from pennylane import numpy as np
from torch.utils.data import TensorDataset, DataLoader

# ==========================================
# 1. Inference (Model Prediction) - 기존 유지
# ==========================================
def get_predictions(model, inputs):
    """Run inference on inputs using the trained model."""
    model.eval()
    with torch.no_grad():
        outputs = model(inputs)
        predicted_labels = torch.argmax(outputs, dim=1)
    return predicted_labels.cpu().numpy()

def data_to_tensor(X, y):
    tensor_X = torch.tensor(X, dtype=torch.complex64)
    tensor_y = torch.tensor(y, dtype=torch.long)
    return tensor_X, tensor_y

t_train_X, t_train_y = data_to_tensor(train_X, train_y)
train_dataset = TensorDataset(t_train_X, t_train_y)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

In [ ]:
import torch
import torch.nn as nn
import pennylane as qml
from pennylane import numpy as np

# ────────────── Quantum Circuit Setup ──────────────

n_qubits = 8
n_layers = 3
dev = qml.device("default.qubit", wires=n_qubits)

def preprocess_to_8d(X):
    """
    X: (N, 256) real-valued
    return: (N, 8) normalized
    """
    X = X.reshape(-1, 8, 32).mean(axis=2)  # 256 → 8
    X = X / np.linalg.norm(X, axis=1, keepdims=True)
    return X


def Quantum_classifier(params):
    """
    Trainable part of the quantum circuit:
    - Rotations + CNOT chain for all layers except last
    - Final layer: only Rotations
    """
    wires = list(range(n_qubits))
    params = params.reshape(-1, 3, n_qubits)

    # All layers except last: Rot + CNOT
    for layer in range(params.shape[0] - 1):
        for i, wire in enumerate(wires):
            qml.Rot(params[layer, 0, i], params[layer, 1, i], params[layer, 2, i], wires=wire)
        for i in range(len(wires) - 1):
            qml.CNOT(wires=[wires[i], wires[i+1]])

    # Last layer: only Rot
    for i, wire in enumerate(wires):
        qml.Rot(params[-1, 0, i], params[-1, 1, i], params[-1, 2, i], wires=wire)

@qml.qnode(dev, interface="torch", diff_method="backprop")
def linear_2qubit_block(state, params):
    """
    state: (n_qubits,) 단일 샘플
    params: (total_params,)
    """
    n_qubits = len(state)           # ✅ batch가 아니면 len(state) 사용
    params = params.reshape(n_layers, 3, n_qubits)

    # StatePrep 또는 AngleEmbedding
    qml.AngleEmbedding(state, wires=range(n_qubits), rotation="Y")

    # Quantum classifier
    for layer in range(n_layers):
        for i in range(n_qubits):
            qml.Rot(params[layer,0,i], params[layer,1,i], params[layer,2,i], wires=i)
        if layer < n_layers - 1:
            for i in range(n_qubits - 1):
                qml.CNOT(wires=[i,i+1])

    return qml.probs(wires=[6,7])  # 4-class 출력




# ────────────── PyTorch QNN Module ──────────────
class QNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.total_params = n_layers*3*n_qubits
        self.params = nn.Parameter(torch.randn(self.total_params)*0.01)

    def forward(self, x):
        # x: (batch_size, 8)
        probs_list = [linear_2qubit_block(x[i], self.params) for i in range(x.shape[0])]
        return torch.stack(probs_list)  # shape: (batch_size, 4)








# ────────────── Model Creation ──────────────

device = "cuda" if torch.cuda.is_available() else "cpu"
model = QNN().to(device)

print("✅ QNN Model Created!")
print(f"• Device: {device}")
print(f"• Total parameters: {model.total_params}")
print(f"• Parameter shape: {model.params.shape}")

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

# ────────────── 가상의 데이터 생성 ──────────────
# 예: 256-dim 입력, 4-class 분류
num_samples = 200
x_data = torch.randn(num_samples, 256)
y_data = torch.randint(0, 4, (num_samples,))

# train/test split
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)
X_train_8d = preprocess_to_8d(X_train.numpy())
X_test_8d = preprocess_to_8d(X_test.numpy())

X_train_t = torch.tensor(X_train_8d, dtype=torch.float32).to(device)
X_test_t = torch.tensor(X_test_8d, dtype=torch.float32).to(device)
y_train_t = y_train.to(device)
y_test_t = y_test.to(device)

# ────────────── 모델, 옵티마이저, 손실함수 ──────────────
model = QNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

# ────────────── 학습 루프 ──────────────
epochs = 1000
best_acc = 0.0
best_params = None

for epoch in range(1, epochs+1):
    model.train()
    optimizer.zero_grad()

    # Forward pass
    outputs = model(X_train_t)
    loss = criterion(outputs, y_train_t)


    # Backpropagation
    loss.backward()

    # Gradient clipping for stability
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()

    # Accuracy 계산
    model.eval()
    with torch.no_grad():
        probs = model(X_test_t)  # ✅ 8차원 입력 사용
        pred = torch.argmax(probs, dim=1)
        acc = (pred == y_test_t).float().mean().item()

    # best_acc, best_params 업데이트
    if acc > best_acc:
        best_acc = acc
        best_params = model.params.detach().clone()

    if epoch % 1 == 0:
        print(f"Epoch [{epoch}/{epochs}] | Loss: {loss.item():.4f} | Test Acc: {acc:.4f} | Best Acc: {best_acc:.4f}")

# ────────────── 학습 종료 후 출력 ──────────────
print("\n✅ Training Finished!")
print(f"Best Test Accuracy: {best_acc:.4f}")



✅ QNN Model Created!
• Device: cpu
• Total parameters: 72
• Parameter shape: torch.Size([72])
Epoch [1/1000] | Loss: 1.4557 | Test Acc: 0.2250 | Best Acc: 0.2250
Epoch [2/1000] | Loss: 1.4550 | Test Acc: 0.2250 | Best Acc: 0.2250
Epoch [3/1000] | Loss: 1.4540 | Test Acc: 0.2250 | Best Acc: 0.2250
Epoch [4/1000] | Loss: 1.4529 | Test Acc: 0.2250 | Best Acc: 0.2250
Epoch [5/1000] | Loss: 1.4516 | Test Acc: 0.2250 | Best Acc: 0.2250
Epoch [6/1000] | Loss: 1.4501 | Test Acc: 0.2250 | Best Acc: 0.2250
Epoch [7/1000] | Loss: 1.4485 | Test Acc: 0.2250 | Best Acc: 0.2250
Epoch [8/1000] | Loss: 1.4467 | Test Acc: 0.2250 | Best Acc: 0.2250
Epoch [9/1000] | Loss: 1.4448 | Test Acc: 0.2250 | Best Acc: 0.2250
Epoch [10/1000] | Loss: 1.4428 | Test Acc: 0.2250 | Best Acc: 0.2250
Epoch [11/1000] | Loss: 1.4406 | Test Acc: 0.2250 | Best Acc: 0.2250
Epoch [12/1000] | Loss: 1.4384 | Test Acc: 0.2250 | Best Acc: 0.2250
Epoch [13/1000] | Loss: 1.4360 | Test Acc: 0.2250 | Best Acc: 0.2250
Epoch [14/1000] | 

In [ ]:
# 배치 하나 가져오기
batch_X, batch_y = next(iter(train_loader))

# 첫 샘플만 사용
x0 = batch_X[0].to(device)

# 회로 시각화
qml.draw_mpl(QNode)(x0, model.params)



In [ ]:
import json

# 1. Specify measurement qubits
measurement = [0, 1]

# 2. Extract trained parameters
params = model.params.detach().cpu().numpy()

# 3. Define circuit for QASM conversion (Ansatz only, no StatePrep or Measurement)
@qml.qnode(dev)
def QASM_Circuit(params):
    # |00000000> 에서 시작 (default)
    QuantumAnsatz(params)
    return qml.probs(wires=measurement_wires)



qasm_str = qml.to_openqasm(
    QASM_Circuit,
    measure_all=False
)(model.params.detach())


submission = {
    "qasm": qasm_str,
    "measurements": measurement_wires
}

with open("baseline.json", "w") as f:
    json.dump(submission, f)

print("✅ baseline.json 생성 완료")


In [ ]:
from google.colab import files
files.download('baseline.json')